# Cleaning data

In [ ]:
import pandas as pd
from tqdm import tqdm
import re

In [ ]:
# read the data
data = pd.read_csv('data/csv/data.csv')

In [ ]:
# seperate the data into two parts according to their VAP (0 or 1) and put them into two variables
data_0 = data[data['VAP'] == 0]
data_1 = data[data['VAP'] == 1]

# print the number of rows of the two variables
print("Class 0: ", data_0.shape[0])
print("Class 1: ", data_1.shape[0])

In [ ]:
# get the values in the TEXT column for each class
text_0 = data_0['TEXT'].values
text_1 = data_1['TEXT'].values

# make a set of all the words in the TEXT column for each class
words_0 = set()
for text in tqdm(text_0):
    for word in text.split():
        words_0.add(word)

words_1 = set()
for text in tqdm(text_1):
    for word in text.split():
        words_1.add(word)
    
# make a set of words that are in both classes
words_both = set()
for word in tqdm(words_0):
    if word in words_1:
        words_both.add(word)

In [ ]:
# print the number of unique words in each class
print("Class 0: ", len(words_0))
print("Class 1: ", len(words_1))
print("Both: ", len(words_both))

In [ ]:
threshold = 0.2
words_to_remove = set()

for word in tqdm(words_both):
    count_0 = 0
    count_1 = 0

    # count the number of times the word appears in each class
    for text in text_0:
        if word in text:
            count_0 += 1
    for text in text_1:
        if word in text:
            count_1 += 1
    
    # calculate the probability of the word appearing in each class
    prob_0 = count_0 / data_0.shape[0]
    prob_1 = count_1 / data_1.shape[0]

    # calculate the ratio of the probability of the word appearing in each class
    ratio = prob_1 / prob_0

    # if the ratio is close to 1 by a certain threshold, remove the word from the set of words that are in both classes
    if ratio >= 1 - threshold and ratio <= 1 + threshold:
        words_to_remove.add(word)

In [ ]:
# print the number of words to remove
print("Words to remove: ", len(words_to_remove))

In [ ]:
print(words_to_remove)

In [ ]:
# Compile a single regular expression that matches any word in words_to_remove
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words_to_remove) + r')\b'
regex = re.compile(pattern)

In [ ]:
def remove_words(text):
    # Replace matched words with a single space
    text = regex.sub(' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# make a copy of the dataframe
print("Making a copy of the dataframe...")
data_cpy = data.copy()

# remove the words from the TEXT column for each row
print("Removing the words from the TEXT column for each row...")
for i in tqdm(range(data_cpy.shape[0])):
    # remove the words from the TEXT column
    text = remove_words(data_cpy.at[i, 'TEXT'])

    # if the new text is empty or contains less than 5 words, remove the row
    if len(text) == 0 or len(text.split()) < 5:
        data_cpy.drop(i, inplace=True)
        continue

    # update the TEXT column
    data_cpy.at[i, 'TEXT'] = text

# save the dataframe to a csv file
print("Saving the dataframe to a csv file...")
data_cpy.to_csv('data/csv/data_cleaned.csv', index=False)

## Cleaned data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/csv/data_cleaned.csv")

In [ ]:
# creating a matrix of word embeddings using the cleaned data and the BioWordVec model
import gensim
import numpy as np

model = gensim.models.KeyedVectors.load_word2vec_format("bin/pubmed.bin", binary=True)

# function to create a matrix of word embeddings
def get_word_embeddings(sample):
    # initialize a matrix of zeros
    embeddings = np.zeros(200)
    # get the words in the sample
    words = sample.split()
    # get the number of words
    num_words = len(words)
    # loop over the words
    for word in words:
        # check if the word is in the model's vocabulary
        if word in model.key_to_index :
            # add the word embedding to the matrix
            embeddings += model[word]
    # return the matrix divided by the number of words
    return np.array(embeddings / num_words)

In [ ]:
# apply the function to the TEXT column, for each embedding, create a new column
df[["embedding_" + str(i) for i in range(200)]] = df["TEXT"].apply(get_word_embeddings).to_list()

# save the dataframe
df.to_csv("data/csv/data_embedded.csv", index=False)

## Processed data

In [ ]:
import pandas as pd
import numpy as np
import tqdm
import re
import ast

In [ ]:
df = pd.read_csv("data/csv/data_embedded.csv")

In [ ]:
# augment the class 1 data using SMOTE
from imblearn.over_sampling import SMOTE

# seperate the features and the labels, the features are the word embeddings, each embedding is a column
X = df[[col for col in df.columns if "embedding" in col]]
y = [int(i) for i in df["VAP"].tolist()]

# initialize the SMOTE object
sm = SMOTE(random_state=42)

# fit the SMOTE object to the data
X_res, y_res = sm.fit_resample(X, y)

# create a dataframe from the augmented data
df_res = pd.DataFrame(X_res, columns=X.columns)
df_res["VAP"] = y_res

# save the dataframe
df_res.to_csv("data/csv/data_resampled.csv", index=False)

In [ ]:
# count the number of samples in each class
df_res["VAP"].value_counts()

# Similarity

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine, euclidean, chebyshev
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data/csv/data_resampled.csv")

In [ ]:
# analyse the similarity between the nodes
import numpy as np
import tqdm

# sim matrices
sim_dic = {
    "cos_sim":np.zeros((len(df), len(df))),
    "euc_dist":np.zeros((len(df), len(df))),
    "cheb_dist":np.zeros((len(df), len(df)))
}

# load the embeddings into a matrix
print("Loading embeddings...")
embeddings = df[[col for col in df.columns if "embedding" in col]].to_numpy()

# add edges to the graph
print("Computing similarities...")
progress = tqdm.tqdm(total=len(df))
for i in range(len(df)):
    for j in range(i+1, len(df)):

        # get the embeddings of the two nodes
        X = embeddings[i]
        Y = embeddings[j]

        # compute the similarity between the two nodes using cosine similarity and euclidean distance and chebyshev distance
        cos_sim = cosine(X, Y)
        euc_dist = euclidean(X, Y)
        cheb_dist = chebyshev(X, Y)

        # save the similarity matrices
        sim_dic["cos_sim"][i, j] = cos_sim
        sim_dic["euc_dist"][i, j] = euc_dist
        sim_dic["cheb_dist"][i, j] = cheb_dist

    # update the progress bar
    progress.update(1)

# close the progress bar
progress.close()

In [ ]:
# save the similarity matrices
np.save("data/sim/cos_sim.npy", sim_dic["cos_sim"])
np.save("data/sim/euc_dist.npy", sim_dic["euc_dist"])
np.save("data/sim/cheb_dist.npy", sim_dic["cheb_dist"])

In [ ]:
# for each similarity matrix, plot the distribution of the similarities
for sim in sim_dic.keys():

    matrix = sim_dic[sim]

    # Flatten the matrix and remove diagonal elements
    values = matrix[np.triu_indices_from(matrix, k=1)]

    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.histplot(values, kde=True, bins=30)
    plt.title('Distribution of ' + sim + ' values')
    plt.xlabel('Similarity Value')
    plt.ylabel('Frequency')

    # Save the plot to a file
    plt.savefig('figures/dist/' + sim + '_distribution.png', dpi=300)
    plt.close()

In [ ]:
# for each similarity matrix, plot the heatmap of the similarities
for sim in sim_dic.keys():

    matrix = sim_dic[sim]

    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.heatmap(matrix, cmap='viridis')
    plt.title('Heatmap of ' + sim + ' values')
    plt.xlabel('Node ID')
    plt.ylabel('Node ID')

    # Save the plot to a file
    plt.savefig('figures/heat/' + sim + '_heatmap.png', dpi=300)
    plt.close()

# Creating Graph

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import tqdm

In [ ]:
df = pd.read_csv("data/csv/data_resampled.csv")
distance_matrix = np.load("data/sim/euc_dist.npy")

In [ ]:
# create a graph
G = nx.Graph()

# add nodes to the graph
for i in tqdm.tqdm(range(len(df))):
    G.add_node(i)

# make the VAP column the node labels
labels = df["VAP"].to_dict()

# add the labels to the graph
nx.set_node_attributes(G, labels, "VAP")

# add edges to the graph
for i in tqdm.tqdm(range(len(df))):

    # get top 200 most similar nodes
    top_200 = np.argsort(distance_matrix[i])[1:201]

    # add edges between the node and the top 200 most similar nodes
    for j in top_200:
        G.add_edge(i, j)

# save the graph to a file
nx.write_gexf(G, "data/graphs/graph.gexf")

In [ ]:
# compute the number of nodes and edges in the graph
print("Number of nodes: ", G.number_of_nodes())
print("Number of edges: ", G.number_of_edges())

# Creating Graph Data

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# load the graph and df
G = nx.read_gexf("data/graphs/graph.gexf")
df = pd.read_csv("data/csv/data_resampled.csv")

In [6]:
# add the node features to the graph
print("Adding node features to the graph...")
for node in tqdm(G.nodes(data=True)):
    
    # get the node id
    node_id = int(node[0])

    # get the node features
    node_features = df.iloc[node_id][[col for col in df.columns if "embedding" in col]].to_numpy()

    # add the node features to the graph
    node[1]["features"] = torch.tensor(node_features, dtype=torch.float)

Adding node features to the graph...


100%|██████████| 5588/5588 [00:02<00:00, 2410.75it/s]


In [7]:
print("Getting the adj matrix...")
adj = torch.tensor(nx.to_numpy_matrix(G))

Getting the adj matrix...


In [8]:
print("Converting the graph to a PyG data object")
data = from_networkx(G)

Converting the graph to a PyG data object


In [9]:
# create trainig, validation and test masks
train_split = 0.8
val_split = 0.1
test_split = 1 - train_split - val_split

# get the number of nodes
num_nodes = data.num_nodes

# Creating indices for splits
indices = np.random.permutation(num_nodes)
train_size = int(train_split * num_nodes)
val_size = int(val_split * num_nodes)

# Create boolean masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# Assigning the masks
train_mask[indices[:train_size]] = True
val_mask[indices[train_size:train_size + val_size]] = True
test_mask[indices[train_size + val_size:]] = True

# add masks to the data
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [10]:
# load the data onto the device
data = data.to(device)

In [11]:
print(data)

Data(edge_index=[2, 2233642], VAP=[5588], label=[5588], features=[5588, 200], id=[2233642], mode='static', num_nodes=5588, train_mask=[5588], val_mask=[5588], test_mask=[5588])


# Model

## GCN

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv

In [18]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels=16, lr=0.001, weight_decay=5e-4):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(200, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2)

        self.lr = lr
        self.weight_decay = weight_decay

        # initialize the optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        # initialize the loss function
        self.criterion = torch.nn.CrossEntropyLoss()
        
        # copy the model to the device
        self.cuda()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

    def fit(self, data, epochs):
        
        # put the model in training mode
        self.train()

        loss = 0

        progress = tqdm(total=epochs)
        for _ in range(epochs):
            out = self(data.features[data.train_mask], data.edge_index)
            loss += self.criterion(out, data.y[data.train_mask]) / G.number_of_nodes()

            # backpropagate the loss
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # compute the training accuracy
            pred = out.argmax(dim=1)
            correct = pred.eq(data.y[data.train_mask]).sum().item()
            train_acc = correct / data.train_mask.sum().item()

            # compute the validation accuracy
            val_out = self(data.features[data.val_mask], data.edge_index)
            pred = val_out.argmax(dim=1)
            correct = pred.eq(data.y[data.val_mask]).sum().item()
            val_acc = correct / data.val_mask.sum().item()

            # update the progress bar
            progress.set_description("Loss: {:.4f}, Train Acc: {:.4f}, Val Acc: {:.4f}".format(loss, train_acc, val_acc))
            progress.update(1)
    
    def test(self, data):
        self.eval()
        out = self(data.features[data.test_mask], data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred.eq(data.y[data.test_mask]).sum().item()
        test_acc = correct / data.test_mask.sum().item()
        return test_acc

In [19]:
# create the model
model = GCN()

# train the model
model.fit(data, 100)

  0%|          | 0/100 [00:58<?, ?it/s]


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
def train(model, optimizer):
    model.train()
    loss = 0

    out = model(data.x, data.edge_index)
    # loss += F.cross_entropy(out[data.train_mask], data.y[data.train_mask]) / G.number_of_nodes()
    loss += loss_function(out[data.train_mask], data.y[data.train_mask]) / G.number_of_nodes()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return float(loss)

In [ ]:
progress_bar = tqdm.tqdm(total=epochs)
loss = 0
for _ in range(epochs):
    loss = train(model, optimizer)
    progress_bar.set_description(f"Loss: {loss}")
    progress_bar.update(1)

progress_bar.close()

model.eval()
correct = 0
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
correct += pred.eq(data.y).sum().item()
print("Accuracy: ", correct / len(data.y))

In [ ]:
from torch_geometric.nn import GATConv


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GATConv(-1, hidden_channels)  # TODO
        self.conv2 = GATConv(hidden_channels, 1)  # TODO

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GAT(hidden_channels=8, heads=8)
print(model)

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.

      loss = criterion(out[data.train_mask].squeeze(), data.y[data.train_mask].float()) / G.number_of_nodes()
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc

In [ ]:
epochs = 100


progress_bar = tqdm.tqdm(total=epochs)
for epoch in range(epochs):
    loss = train()
    val_acc = test(data.train_mask)
    
    progress_bar.set_description(f"Loss: {loss} Val Acc: {val_acc}")
    progress_bar.update(1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')


# analyse the similarity between the nodes
similarity = np.zeros((len(df), len(df)))

# add edges to the graph
for i in tqdm.tqdm(range(len(df))):

    for j in range(len(df)):   

        # check if the two nodes are the same
        if i != j:

            X = df.iloc[i]["EMBEDDINGS"]
            Y = df.iloc[j]["EMBEDDINGS"]

            # convert the strings to numpy arrays
            X = np.fromstring(X[1:-1], sep=" ")
            Y = np.fromstring(Y[1:-1], sep=" ")

            # compute the similarity between the two nodes
            s = np.dot(X, Y)

            # add the value to the similarity matrix
            similarity[i, j] = s

# returns the mean of the similarity matrix
print("Mean similarity: ", similarity.mean())

# returns the maximum value of the similarity matrix
print("Maximum similarity: ", similarity.max())

# returns the minimum value of the similarity matrix
print("Minimum similarity: ", similarity.min())

# returns the standard deviation of the similarity matrix
print("Standard deviation: ", similarity.std())

# returns the variance of the similarity matrix
print("Variance: ", similarity.var())

# returns the median of the similarity matrix
print("Median: ", np.median(similarity))

# returns the 25th percentile of the similarity matrix
print("25th percentile: ", np.percentile(similarity, 25))

# returns the 75th percentile of the similarity matrix
print("75th percentile: ", np.percentile(similarity, 75))

In [ ]:
# save the similarity matrix
np.save("similarity.npy", similarity)